<a href="https://colab.research.google.com/github/Igor-Belskiy/rbk-case/blob/main/%D0%A0%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B5_%D0%91%D0%B5%D0%BB%D1%8C%D1%81%D0%BA%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
import numpy as np   # для работы с данными
import pandas as pd   # для работы с таблицами
import time    # Импортируем библиотеку time
import re

from tensorflow.keras.preprocessing.text import Tokenizer   # Методы для работы с текстами и преоборазвания их в последовательности
from tensorflow.keras import utils  # для работы с категориальными данными

# Загрузка данных

In [28]:
from google_drive_downloader import GoogleDriveDownloader as gdd   # для загрузки с гугл-драйва в колабораторию

# Копию бучающег и тестовог датасета предварительно загрузил к себе на гугл-диск
# и Колабораторию загружаю их напрямую уже со своего диска (досутуп отркыт для всех по ссылке)

gdd.download_file_from_google_drive(file_id='1OJs8ZLeKe1BflJMGWM5QLbsbMHCiWhJ7', 
                                    dest_path='/content/test_dataset_test.csv', 
                                    showsize=True,
                                    overwrite=True) #Скачиваем тестовый датасет

gdd.download_file_from_google_drive(file_id='1sIbebBPuT3YC6OC-uUP8Q7i3YUJfS2Ad', 
                                    dest_path='/content/train_dataset_train.csv', 
                                    showsize=True,
                                    overwrite=True) #Скачиваем тренировочный датасет

1.2 MiB Done.
2.9 MiB Done.


In [29]:
df_test = pd.read_csv('/content/test_dataset_test.csv', index_col= 0)

df_train = pd.read_csv('/content/train_dataset_train.csv', index_col= 0)

In [30]:
df_train.columns

Index(['title', 'publish_date', 'session', 'authors', 'ctr', 'category',
       'tags', 'views', 'depth', 'full_reads_percent'],
      dtype='object')

In [108]:
df_train.shape[0]

7000

# Предварительный осмотр данных

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7000 entries, 624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ to 627f1c089a794743b070ff73hVvdVmFxS2SlZ2_lECDEow
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               7000 non-null   object 
 1   publish_date        7000 non-null   object 
 2   session             7000 non-null   object 
 3   authors             7000 non-null   object 
 4   ctr                 7000 non-null   float64
 5   category            7000 non-null   object 
 6   tags                7000 non-null   object 
 7   views               7000 non-null   int64  
 8   depth               7000 non-null   float64
 9   full_reads_percent  7000 non-null   float64
dtypes: float64(3), int64(1), object(6)
memory usage: 601.6+ KB


In [ ]:
df_train.head()

,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent
document_id,,,,,,,,,,
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,Европейский банк развития приостановил доступ ...,2022-04-04 10:29:44,IDE7mtH4RBqGn-8MXfGffQ,[],1.580,5409f11ce063da9c8b588a18,"['55928d339a794751dc8303d6', '542d1e28cbb20f86...",20460,1.134,35.850
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,Кремль назвал регулярным процессом учебные зап...,2022-02-18 10:00:39,KtVJsteHStO5oditt3Uvzw,"['54244e01cbb20f03076b236d','5878a2ec9a7947e53...",1.853,5409f11ce063da9c8b588a12,"['549d25df9a794775979561d2', '58abcf539a7947f1...",19038,1.142,38.355
620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA,Госсекретарь Швеции заявила о нежелании вступа...,2022-02-12 04:24:02,hk7puWJwSziw0m3sfTkKWA,[],0.000,5409f11ce063da9c8b588a12,"['5430f451cbb20f73931ecd05', '5409f15de063daa0...",51151,1.185,36.424
6262a5889a79470b78c9ca307UKY2SSZTjCcjhwBzxw37w,Песков назвал прагматичной выдачу лицензии Газ...,2022-04-22 13:24:55,7UKY2SSZTjCcjhwBzxw37w,[],0.000,5409f11ce063da9c8b588a12,"['5409f297e063daa0f408b11c', '545caa9ecbb20f36...",3782,1.053,30.169
626678929a79477ca0101568wuMYES90REuV5YhrN75IXg,В Хабаровске задержали главу филиала РАНХиГС п...,2022-04-25 10:42:23,wuMYES90REuV5YhrN75IXg,[],0.000,5433e5decbb20f277b20eca9,"['5409f42ae063daa0f408b5d7', '585c20e19a79470e...",3065,1.063,34.617


In [ ]:
df_test.head()

,title,publish_date,session,authors,ctr,category,tags
document_id,,,,,,,
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw,«Крайне провокационная тема»: как Россия и США...,2022-02-01 17:02:44,J0AvX96vTAaQCiWVbzoMdw,[],0.000,5409f11ce063da9c8b588a12,"['58ca77679a794700b1e37f0a', '5409f1b7e063daa0..."
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw,Власти Херсонской области пообещали сделать ру...,2022-05-24 00:50:55,QS5CqzXYRnmDdR2LaSreEw,[],1.598,5409f11ce063da9c8b588a12,"['5433603acbb20f6e5def0cc5', '5409f420e063daa0..."
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw,Премьер и президент Финляндии призвали срочно ...,2022-05-12 07:11:51,XVsoyrUOT8OJJg2_finJhw,[],2.006,5409f11ce063da9c8b588a12,"['5409f270e063daa0f408b0a7', '5409f196e063daa0..."
628618629a7947d4927eb812upfii3whSSuMXCqcqF8VbQ,В Госдуму внесли проект о временных администра...,2022-05-19 11:54:35,upfii3whSSuMXCqcqF8VbQ,[],0.000,5409f11ce063da9c8b588a12,"['5409f145e063daa0f408ad35', '5409f30fe063daa0..."
620e76109a7947235623695b5hzCiIHdSYKQIr8WAM18bw,Блинкен заявил в ООН о готовности России начат...,2022-02-17 16:28:55,5hzCiIHdSYKQIr8WAM18bw,['604f2c199a79478ec0029eae'],9.136,5409f11ce063da9c8b588a12,"['5409f15de063daa0f408ada8', '545d4dcdcbb20fbc..."


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000 entries, 61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw to 6236f9129a79477b0ef18ae0ewByZQ1FQBK2dpXyY77rYw
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         3000 non-null   object 
 1   publish_date  3000 non-null   object 
 2   session       3000 non-null   object 
 3   authors       3000 non-null   object 
 4   ctr           3000 non-null   float64
 5   category      3000 non-null   object 
 6   tags          3000 non-null   object 
dtypes: float64(1), object(6)
memory usage: 187.5+ KB


In [ ]:
# Смотрим, сколько каких значений в столбцах

for x in df_train.columns:
    print('Столбец: ', x)
    print(df_train[x].value_counts())
    print('\n')

Столбец:  title
Какие места на Украине взяли под контроль российские военные. Карта                                                                                                                10
Как менялась ключевая ставка ЦБ за последние годы. Инфографика                                                                                                                      7
Как Запад и Россия поставили рекорд по высылкам дипломатов. Инфографика                                                                                                             6
Какие приграничные пункты попали под обстрел со стороны Украины. Карта                                                                                                              5
Какие регионы объявили высокий уровень террористической опасности. Карта                                                                                                            4
                                                                          

In [ ]:
df_train['title'].value_counts()

Какие места на Украине взяли под контроль российские военные. Карта                                                                                                                10
Как менялась ключевая ставка ЦБ за последние годы. Инфографика                                                                                                                      7
Как Запад и Россия поставили рекорд по высылкам дипломатов. Инфографика                                                                                                             6
Какие приграничные пункты попали под обстрел со стороны Украины. Карта                                                                                                              5
Какие регионы объявили высокий уровень террористической опасности. Карта                                                                                                            4
                                                                                          

In [ ]:
for i in df_train.columns :  
  print(f'Колонка "{i}": \nУникальныех значении: {df_train[i].nunique()} \n {df_train[i].unique()}\n\n') 

Колонка "title": 
Уникальныех значении: 6954 
 ['Европейский банк развития приостановил доступ Москвы и Минска к финансам'
 'Кремль назвал регулярным процессом учебные запуски баллистических ракет'
 'Госсекретарь Швеции заявила о нежелании вступать в НАТО из-за Украины\n                \n                                                    \n\n    \n\n    Политика,\xa007:24'
 ... 'Брюс Уиллис завершит актерскую карьеру из-за болезни'
 'Экс-посол Франции назвал болезненным для Европы отказ от российской нефти\n                \n                                                    \n\n    \n\n    Экономика,\xa002:15'
 'Минфин США предостерег зарубежные банки от помощи России в обходе санкций']


Колонка "publish_date": 
Уникальныех значении: 6985 
 ['2022-04-04 10:29:44' '2022-02-18 10:00:39' '2022-02-12 04:24:02' ...
 '2022-03-30 21:59:02' '2022-05-04 23:15:48' '2022-05-14 03:41:33']


Колонка "session": 
Уникальныех значении: 5901 
 ['IDE7mtH4RBqGn-8MXfGffQ' 'KtVJsteHStO5oditt3Uvzw'
 'hk

# Готовим колонку title методом Bag of Words

In [121]:

maxWordsCount = 15000   # Определяем максимальное количество слов/индексов, учитываемое при обучении текстов

# Воспользуемся встроенной в Keras функцией Tokenizer для разбиения текста и превращения в матрицу числовых значений
# num_words=maxWordsCount - определяем максимальное количество слов/индексов, учитываемое при обучении текстов
# filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n' - избавляемся от ненужных символов
# lower=True - приводим слова к нижнему регистру
# split=' ' - разделяем слова по пробелу
# char_level=False - токенизируем по словам (Если будет True - каждый символ будет рассматриваться как отдельный токен )
tokenizer = Tokenizer(num_words=maxWordsCount, 
                      filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', 
                      lower=True, 
                      split=' ', 
                      oov_token='unknown', 
                      char_level=False)


In [122]:
# Перед обучением частотного словаря удаляем из него отметки минут в тексте некоторых заголовков.
# Так как стандартный фильтр токенайзера их не удаляет

titles_for_tokenizer = [re.sub(r"\d{2}:\d{2}", " ", df_train['title'][x]) for x in range(df_train.shape[0])]

In [123]:

tokenizer.fit_on_texts(titles_for_tokenizer) # "Скармливаем" наши тексты, т.е. даём в обработку методу, который соберет словарь частотности

items = list(tokenizer.word_index.items()) # Вытаскиваем индексы слов для просмотра


In [124]:
items[:20]      # Смотрим 20 самых популярных слов частотного словаря

[('unknown', 1),
 ('в', 2),
 ('о', 3),
 ('политика', 4),
 ('на', 5),
 ('и', 6),
 ('с', 7),
 ('россии', 8),
 ('общество', 9),
 ('из', 10),
 ('за', 11),
 ('по', 12),
 ('заявил', 13),
 ('об', 14),
 ('сша', 15),
 ('украине', 16),
 ('что', 17),
 ('для', 18),
 ('украины', 19),
 ('не', 20)]

In [125]:
len(items)    # Количество слов в частотном словаре токенайзера

14465

In [126]:
# Преобразовываем текст в последовательность индексов согласно частотному словарю

train_title = []

train_title = tokenizer.texts_to_sequences(df_train['title'])

In [127]:
# Осматриваем результаты полученных промежуточных преоборазовании

print("Взглянем на фрагмент обучающего текста:")
print("В виде оригинального текста:              ", df_train['title'][10])
print("Он же в виде последовательности индексов: ", train_title[10], '\n')

Взглянем на фрагмент обучающего текста:
В виде оригинального текста:               Медведев обвинил Зеленского в выдвижении «невыполнимых ультиматумов»
Он же в виде последовательности индексов:  [100, 137, 126, 2, 6052, 6053, 6054] 



In [128]:
len(train_title)

7000